In [29]:
from caffe.proto import caffe_pb2
import numpy as np
import lmdb
import os
import cv2
from random import shuffle
from progressbar import ProgressBar

In [4]:
N = len(os.listdir('imgs'))
int(N*0.05)

594

In [2]:
type(os.listdir('imgs'))

list

In [6]:
im = cv2.imread('imgs/good-465.png')

In [7]:
np.shape(im)

(288, 432, 3)

In [9]:
def transform(im, width=256, height=256):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # Histgram Equalization
    im[:, :, 0] = cv2.equalizeHist(im[:, :, 0])
    im[:, :, 1] = cv2.equalizeHist(im[:, :, 1])
    im[:, :, 2] = cv2.equalizeHist(im[:, :, 2])
    # Resize
    im = cv2.resize(im, (width, height), interpolation=cv2.INTER_CUBIC)
    return im

In [23]:
def create_datum(im, label):
    datum = caffe_pb2.Datum(
        channels=3,
        width=256,
        height=256,
        label=label,
        data=im.tostring())
    return datum

In [32]:
def create_lmdb():
    imgs = os.listdir('imgs')
    shuffle([img for img in imgs if 'png' not in img])
    div = int(len(imgs)*0.1)
    valid = imgs[0:div]
    train = imgs[div:]
    tr_db = lmdb.open('train_data', map_size=1e10)
    with tr_db.begin(write=True) as db_txn:
        bar = ProgressBar()
        for im_name in bar(train):
            im = cv2.imread('imgs/'+im_name)
            try:
                im = transform(im)
            except:
                continue
            if 'good' in im_name:
                datum = create_datum(im,0)
            else:
                datum = create_datum(im,0)
            db_txn.put(im_name, datum.SerializeToString())
    tr_db.close()
    va_db = lmdb.open('valid_data', map_size=1e10)
    with va_db.begin(write=True) as db_txn:
        bar = ProgressBar()
        for im_name in bar(valid):
            im = cv2.imread('imgs/'+im_name)
            try:
                im = transform(im)
            except:
                continue
            if 'good' in im_name:
                datum = create_datum(im,0)
            else:
                datum = create_datum(im,0)
            db_txn.put(im_name, datum.SerializeToString())
    va_db.close()

In [33]:
create_lmdb()

100% (10708 of 10708) |###################| Elapsed Time: 0:01:30 Time: 0:01:30
100% (1189 of 1189) |#####################| Elapsed Time: 0:00:10 Time: 0:00:10
